In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import KFold
from tensorflow.keras.utils import plot_model
import scikitplot
from matplotlib import pyplot

In [ ]:
# Set Seed
import random
np.random.seed(11)
# set_seed(11)
random.seed(11)

In [ ]:
df = pd.read_csv('../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import seaborn as sns
from keras import models
from keras import layers
from keras import optimizers


**Type of data**

In [ ]:
df.columns


In [ ]:
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

**Image Seperation**

In [ ]:
df["emotion"].value_counts()

In [ ]:
train = df.loc[df['Usage']=='Training']
print('train:'+ str(train.shape))
privateTest = df.loc[df['Usage']=='PrivateTest']
print('privateTest:'+ str(privateTest.shape))
publicTest = df.loc[df['Usage']=='PublicTest']
print('train:'+ str(publicTest.shape))

In [ ]:
train_labels = df["emotion"]
train_labels = to_categorical(train_labels)
train_labels

**Change ' ' to list ( , , )**
> df["pixels"].str.split(" ").tolist()

1. To make the labels be categorical

 - Converts a class vector (integers) to binary class matrix.


2. To reshape img to be 48* 48 type float32 = 0-1

In [ ]:
train.shape

In [ ]:
df

In [ ]:
img_labels = df["emotion"]
img_labels = to_categorical(img_labels)

img_pixels = df["pixels"].str.split(" ").tolist()
img_pixels = np.uint8(img_pixels)
img_pixels = img_pixels.reshape((35887, 48, 48, 1))
img_pixels = img_pixels.astype("float32") / 255

In [ ]:
train_labels = train["emotion"]
train_labels = to_categorical(train_labels)

train_pixels = train["pixels"].str.split(" ").tolist()
train_pixels = np.uint8(train_pixels)
train_pixels = train_pixels.reshape((28709, 48, 48, 1))
train_pixels = train_pixels.astype("float32") / 255

In [ ]:
private_labels = privateTest["emotion"]
private_labels = to_categorical(private_labels)

private_pixels = privateTest["pixels"].str.split(" ").tolist()
private_pixels = np.uint8(private_pixels)
private_pixels = private_pixels.reshape((3589, 48, 48, 1))
private_pixels = private_pixels.astype("float32") / 255

In [ ]:
public_labels = publicTest["emotion"]
public_labels = to_categorical(public_labels)

public_pixels = publicTest["pixels"].str.split(" ").tolist()
public_pixels = np.uint8(public_pixels)
public_pixels = public_pixels.reshape((3589, 48, 48, 1))
public_pixels = public_pixels.astype("float32") / 255

In [ ]:
train_pixels.shape

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img_pixels, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.1, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
plt.figure(0, figsize=(12,6))
# for i in range(1, 13):
#     plt.subplot(3,4,i)
#     plt.imshow(train_pixels[i, :, :, 0], cmap="gray")
plt.imshow(train_pixels[0,:48,:48,0], cmap="gray")
plt.show()
train_labels[0]



In [ ]:
fig = pyplot.figure(1, (14, 14))

k = 0
for label in sorted(df.emotion.unique()):
    for j in range(7):
        px = df[df.emotion==label].pixels.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

        k += 1
        ax = pyplot.subplot(7, 7, k)
        ax.imshow(px, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotion_labels[label])
        pyplot.tight_layout()

K-fold

In [ ]:
# Model configuration
batch_size = 32
loss_function = "categorical_crossentropy"
no_classes = 7 # 7 emotions
no_epochs = 50
optims = optimizers.Nadam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam')
optimizer = optims
verbosity = 1
num_folds = 10
input_shape = (48, 48, 1)



In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

In [ ]:
batch_size = 64 #batch size of 32 performs the best.
epochs = 100
optims = [
    optimizers.Nadam(learning_rate=0.0006, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
    optimizers.Adam(0.001),
]

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)
train_datagen.fit(X_train)

In [ ]:

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for i, j in kfold.split(X_train, y_train):
    net = models.Sequential(name="DCNN")
    net.add(
    Conv2D(
            filters=64,
            kernel_size=(5,5),
    #         input_shape=(img_width, img_height, img_depth),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_1'))
    net.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_2'
        )
    )
    net.add(BatchNormalization(name='batchnorm_2'))
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
    net.add(Dropout(0.4, name='dropout_1'))
    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_3'
        )
    )
    net.add(BatchNormalization(name='batchnorm_3'))
    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_4'
        )
    )
    net.add(BatchNormalization(name='batchnorm_4'))
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
    net.add(Dropout(0.4, name='dropout_2'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_5'
        )
    )
    net.add(BatchNormalization(name='batchnorm_5'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_6'
        )
    )
    net.add(BatchNormalization(name='batchnorm_6'))
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
    net.add(Dropout(0.5, name='dropout_3'))
    net.add(Flatten(name='flatten'))
    net.add(
        Dense(
            128,
            activation='elu',
            kernel_initializer='he_normal',
            name='dense_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_7'))
    net.add(Dropout(0.6, name='dropout_4')) # 60% to shut the neural and change in order to improve the output
    net.add(
        Dense(7,
            activation='softmax',
            name='out_layer'
        )
    )
     # Compile the model
    net.compile(loss=loss_function,
                  optimizer=optims[0],
                  metrics=['accuracy']) 
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'training for fold {fold_no} ...')   
    # Fit data to model
    history = net.fit(X_train[i], y_train[i],
                batch_size=batch_size,
                epochs=no_epochs,
                verbose=verbosity)  
    # Generate generalization metrics
    scores = net.evaluate(X_train[j], y_train[j], verbose=0)
    print(f'Score for fold {fold_no}: {net.metrics_names[0]} of {scores[0]}; {net.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0]) 
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

**Add machine learning model**

In [ ]:
def build_net(optims):
    # Set Seed
    np.random.seed(11)
    random.seed(11)
    net = models.Sequential(name="DCNN")
    net.add(
    Conv2D( #batch size,h,w,d : batch size is no. of img
            filters=64, # change depth(channel) from 1 to 64
            kernel_size=(5,5), #kernel that we use to filter img
    #         input_shape=(img_width, img_height, img_depth),
            activation='elu', #function used to get output of the node
            padding='same', # 
            kernel_initializer='he_normal',
            name='conv2d_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_1'))
    net.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_2'
        )
    )
    net.add(BatchNormalization(name='batchnorm_2'))
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
    net.add(Dropout(0.4, name='dropout_1'))
    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_3'
        )
    )
    net.add(BatchNormalization(name='batchnorm_3'))
    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_4'
        )
    )
    net.add(BatchNormalization(name='batchnorm_4'))
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
    net.add(Dropout(0.4, name='dropout_2'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_5'
        )
    )
    net.add(BatchNormalization(name='batchnorm_5'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_6'
        )
    )
    net.add(BatchNormalization(name='batchnorm_6'))
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
    net.add(Dropout(0.5, name='dropout_3'))
    net.add(Flatten(name='flatten'))
    net.add(
        Dense(
            128,
            activation='elu',
            kernel_initializer='he_normal',
            name='dense_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_7'))
    net.add(Dropout(0.6, name='dropout_4')) # 60% to shut the neural and change in order to improve the output
    net.add(
        Dense(7,
            activation='softmax',
            name='out_layer'
        )
    )
    net.compile(
        loss='categorical_crossentropy',
        optimizer=optims,
        metrics=['accuracy']
    )
    net.build((35887, 48, 48, 1))

    net.summary()
    return net

In [ ]:

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau( #Reduce learning rate when a metric has stopped improving.
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
#     early_stopping,
    lr_scheduler,
]

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)
train_datagen.fit(X_train)

In [ ]:
batch_size = 64 #batch size of 32 performs the best.
epochs = 100
optims = [
    optimizers.Nadam(learning_rate=0.0006, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
    optimizers.Adam(0.001),
]


# I tried both `Nadam` and `Adam`, the difference in results is not different but I finally went with Nadam as it is more popular.
net = build_net(optims[1]) 
print(net)
history = net.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_valid, y_valid),
    steps_per_epoch=len(X_train) / batch_size,
    epochs=epochs,
    callbacks=callbacks,
    use_multiprocessing=True
)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, "g", label = "Training acc")
plt.plot(epochs, val_acc, "r", label = "Validation acc")
plt.title("Accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "g", label = "Training Loss")
plt.plot(epochs, val_loss, "r", label = "Validation Loss")
plt.title("Loss")

plt.legend()
plt.show()

One Epoch is when an ENTIRE dataset is passed forward and backward through the neural network only ONCE.

Since one epoch is too big to feed to the computer at once we divide it in several smaller batches.

# **I reduced *batch_size* and *epochs* , increase *learning_rate* when the model seems to stuck at 0.69 val_accuracy** 

Arguments

learning_rate: A Tensor or a floating point value. The learning rate.

beta_1: A float value or a constant float tensor. The exponential decay rate for the 1st moment estimates.

beta_2: A float value or a constant float tensor. The exponential decay rate for the exponentially weighted infinity norm.

epsilon: A small constant for numerical stability.

name: Optional name for the operations created when applying gradients. Defaults to "Nadam".

In [ ]:
batch_size = 32 #batch size of 32 performs the best.
epochs = 50
optims = optimizers.Nadam(learning_rate=0.0009, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam')


Save model

In [ ]:
net.save('../input/my_facial_expression_reg_0.69')


In [ ]:
net.compile(optimizer = optims, loss = "categorical_crossentropy", metrics = ["accuracy"])

history_2 = net.fit(X_train, y_train, batch_size = batch_size, epochs = epochs,
                validation_data = (X_valid, y_valid))

In [ ]:
net.save('../output/my_facial_expression_reg')

In [ ]:
yhat_train = net.predict_classes(X_train)
scikitplot.metrics.plot_confusion_matrix(np.argmax(y_train, axis=1), yhat_train, figsize=(7,7))

print(f'total wrong validation predictions: {np.sum(np.argmax(y_train, axis=1) != yhat_train)}\n\n')


In [ ]:
yhat_valid = net.predict_classes(X_valid)
scikitplot.metrics.plot_confusion_matrix(np.argmax(y_valid, axis=1), yhat_valid, figsize=(7,7))

print(f'total wrong validation predictions: {np.sum(np.argmax(y_valid, axis=1) != yhat_valid)}\n\n')



The result by confusion matrix is ok but can be better by difference model.

In [ ]:
yhat_valid = net.predict_classes(public_pixels)
scikitplot.metrics.plot_confusion_matrix(np.argmax(public_labels, axis=1), yhat_valid, figsize=(7,7))

print(f'total wrong validation predictions: {np.sum(np.argmax(public_labels, axis=1) != yhat_valid)}\n\n')

In [ ]:
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

The confusion matrix shows that the model is doing good but it's performance is low on class Fear, Sad, and Neutral. One of the reason for this issue may happen because the images are hard to identify the emotion even by human. If I reduce the number of class the result will be better but for the whole 7 classes, I got the best accuracy at 70%.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(y_valid, axis=1), yhat_valid))

In [ ]:
n = net.predict_classes(train_pixels)
t = train_labels

Test our image


In [ ]:
import cv2
import numpy as np

In [ ]:
img = cv2.imread('../input/mind-angry/angrymind.jpg',0)
img = cv2.resize(img,(48,48))
img = np.reshape(img,[1,48,48,1])
img_pixels = img.astype("float32") / 255
classes = net.predict_classes(img)
# print classes

In [ ]:
emotion_labels[classes[0]]